Лабораторная работа
=====================

Цель лабораторной работы получить практические знания по работе с признаками на известном датасете статистики самоубийств.

Вам необходимо будет подготовить данные для обучения линейной модели предсказания количества самоубийств (столбец - suicides/100k pop).

Чек-лист:
0. Изучите файл annotation.txt. Там содержится информация о датасете.
1. Загрузите датасет data.csv.
2. Посмотрите на данные. Отобразите общую информацию по признакам (вспомните о describe и info). Напишите в markdown свои наблюдения.
3. Выявите пропуски, а также возможные причины их возникновения. Решите, что следует сделать с ними. Напишите в markdown свои наблюдения.
4. Оцените зависимости переменных между собой. Используйте корреляции. Будет хорошо, если воспользуетесь profile_report. Напишите в markdown свои наблюдения.
5. Определите стратегию преобразования категориальных признаков (т.е. как их сделать адекватными для моделей).
6. Найдите признаки, которые можно разделить на другие, или преобразовать в другой тип данных. Удалите лишние, при необходимости.
7. Разделите выборку на обучаемую и тестовую.
8. Обучите линейную модель. Напишите в markdown свои наблюдения по полученным результатам.

Если возникнут затруднения, то смотрите на материал практических занятий. Данного там должно хватить для выполнения всех пунктов. Желаю успеха!

In [209]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import GradientBoostingClassifier as gb
from sklearn.metrics import roc_auc_score as roc, log_loss
from sklearn.linear_model import LinearRegression

In [210]:
data = pd.read_csv("data.csv")
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [211]:
data['gdp_for_year'] = data[' gdp_for_year ($) '].apply(lambda x: int(''.join(s for s in x if s.isdigit()))).astype(int) 

data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,gdp_for_year
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,2156624900
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,2156624900
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,2156624900


## преобразовал столбец ВВП к числовому значению, почему-то регрессия видела данный столбец как строку.

In [212]:
data.describe()


,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($),gdp_for_year
count,27820.000000,2.782000e+04,27820.000000,8364.000000,27820.000000,2.782000e+04
mean,242.574407,1.844794e+06,12.816097,0.776601,16866.464414,4.455810e+11
std,902.047917,3.911779e+06,18.961511,0.093367,18887.576472,1.453610e+12
min,0.000000,2.780000e+02,0.000000,0.483000,251.000000,4.691962e+07
25%,3.000000,9.749850e+04,0.920000,0.713000,3447.000000,8.985353e+09
50%,25.000000,4.301500e+05,5.990000,0.779000,9372.000000,4.811469e+10
75%,131.000000,1.486143e+06,16.620000,0.855000,24874.000000,2.602024e+11
max,22338.000000,4.380521e+07,224.970000,0.944000,126352.000000,1.812071e+13


In [213]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
gdp_for_year          27820 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 2.3+ MB


## Так как времени мало, изучать пропуски в столбце "HDI for year" не захотел и дропнем в конце.
## Надо распарсить страну и год на 2 столбца, категориальные атрибуты надо заванхотить для линейной модели, а страны попробую заменить таргет энкодингом

In [214]:
data.columns

Index(['sex', 'age', 'suicides_no', 'population', 'suicides/100k pop',
       'country-year', 'HDI for year', ' gdp_for_year ($) ',
       'gdp_per_capita ($)', 'generation', 'gdp_for_year'],
      dtype='object')

In [216]:
data['year']=data['country-year'].str.extract('(\d+)').astype(int) 
data['country']=data['country-year'].str.extract(r'(^\w{4})').astype(object) 


## Распарсил, как смог, нормально выделить страну через extract не получилось.

In [217]:
data['country'].value_counts().head(5)

Unit    816
Aust    742
Sain    672
Slov    516
Turk    432
Name: country, dtype: int64

In [218]:
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

In [219]:
data['country_targ_enc'] = calc_smooth_mean(data, by='country', on='suicides/100k pop', m=20)

## Таргет энкодинг на страны на подобии скользящего среднего, только непонятно, что делать, если такой страны не будет в тренировочном наборе, но будет в тестовом, как быть с таргет энкодингом в таком случае непонятно

In [220]:
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,gdp_for_year,year,country,country_targ_enc
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900,1987,Alba,4.158739
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,2156624900,1987,Alba,4.158739
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900,1987,Alba,4.158739
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,2156624900,1987,Alba,4.158739
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,2156624900,1987,Alba,4.158739


### Куча ванхот энкодингов

In [221]:
df_g = pd.get_dummies(data['sex'], prefix = 'sex')
data = pd.concat([data, df_g], axis=1)
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,gdp_for_year,year,country,country_targ_enc,sex_female,sex_male
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900,1987,Alba,4.158739,0,1
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,2156624900,1987,Alba,4.158739,0,1
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,2156624900,1987,Alba,4.158739,1,0
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,2156624900,1987,Alba,4.158739,0,1
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,2156624900,1987,Alba,4.158739,0,1


In [222]:
df_g = pd.get_dummies(data['age'], prefix = 'age')
data = pd.concat([data, df_g], axis=1)
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,...,country,country_targ_enc,sex_female,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,...,Alba,4.158739,0,1,1,0,0,0,0,0
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,...,Alba,4.158739,0,1,0,0,1,0,0,0
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,...,Alba,4.158739,1,0,1,0,0,0,0,0
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,...,Alba,4.158739,0,1,0,0,0,0,0,1
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,...,Alba,4.158739,0,1,0,1,0,0,0,0


In [223]:
df_g = pd.get_dummies(data['generation'], prefix = 'generation')
data = pd.concat([data, df_g], axis=1)
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,...,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,...,0,0,0,0,0,0,1,0,0,0
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,...,1,0,0,0,0,0,0,0,0,1
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,...,0,0,0,0,0,0,1,0,0,0
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,...,0,0,0,1,0,1,0,0,0,0
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,...,0,0,0,0,1,0,0,0,0,0


In [224]:
data.drop(['sex','age', 'suicides_no','generation','HDI for year','country','country-year',' gdp_for_year ($) '], axis=1, inplace=True)

In [235]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 20 columns):
population                    27820 non-null int64
suicides/100k pop             27820 non-null float64
gdp_per_capita ($)            27820 non-null int64
gdp_for_year                  27820 non-null int64
year                          27820 non-null int64
country_targ_enc              27016 non-null float64
sex_female                    27820 non-null uint8
sex_male                      27820 non-null uint8
age_15-24 years               27820 non-null uint8
age_25-34 years               27820 non-null uint8
age_35-54 years               27820 non-null uint8
age_5-14 years                27820 non-null uint8
age_55-74 years               27820 non-null uint8
age_75+ years                 27820 non-null uint8
generation_Boomers            27820 non-null uint8
generation_G.I. Generation    27820 non-null uint8
generation_Generation X       27820 non-null uint8
generation_Generatio

## Готовим данные для модели

In [226]:
df=data.copy()
Y=data['suicides/100k pop'].values

X=df.drop('suicides/100k pop',axis=1).values


In [227]:
X_train, X_test, y_train, y_test = tts(X,Y,
                                                     train_size = 0.8,
                                                     random_state = 42
                                                     )

/home/david/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [231]:
X_train

array([[2.31000000e+06, 6.40400000e+03, 1.89719984e+11, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.75210000e+05, 1.70400000e+03, 8.73465381e+09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.05083000e+05, 3.76500000e+03, 2.87062464e+09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.60187400e+06, 7.91800000e+03, 2.87530508e+11, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.06880000e+04, 7.80600000e+03, 2.58817606e+09, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.10670000e+06, 1.64440000e+04, 6.12939685e+11, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

### Остановился на обучении регрессии, что-то пошло не так, где-то тип не тот. Если есть возможность у Артура на проверке быстро дописать код, интересно, какой скор выйдет :)

In [228]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').